# 🔬 Data exploration and preparation
In this notebook, we'll examine the dataset and create a subset of it for further analysis. The dataset was relatively clean when downloaded, though we addressed some problematic delimiter issues for you. If you're interested in tackling these issues firsthand, the original dataset is available at the [Book-Crossing Dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/).

### 1. Loading the data
Load the three datasets and explore the data.


In [9]:
import pandas as pd
ratings = pd.read_csv('data/BX-Book-Ratings.csv', delimiter=';')
books = pd.read_csv('data/BX-Books.csv', delimiter=';', encoding='latin-1', low_memory=False)
users = pd.read_csv('data/BX-Users.csv', delimiter=';', encoding='latin-1')

In [10]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


### 2. Cleaning the data
Ensure that all reviews are linked to a book. Investigate whether there are any reviews that lack a corresponding book or user. Verify the accuracy of author names and identify any anomalies, such as users who have submitted an unusually high number of reviews. Describe the process you followed to clean the data.

In [11]:
# Check null values in ratings
ratings.isnull().sum(axis=0)

# Check number of reviews per user (what should be done with this?)
ratings['User-ID'].value_counts().sort_values(ascending=False)

# Check if all books exist in ratings -> no, so remove these ratings
#ratings = ratings[ratings['ISBN'].isin(books['ISBN'])]

# Check if all users exist in ratings -> yes
ratings[ratings['User-ID'].isin(users['User-ID'])]

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149774,276704,1563526298,9
1149775,276706,0679447156,0
1149776,276709,0515107662,10
1149777,276721,0590442449,10


### 3. Subsetting the data
The publication accompanied with this dataset [Improving Recommendation Lists Through Topic Diversification](http://www2.informatik.uni-freiburg.de/~cziegler/BX/WWW-2005-Preprint.pdf) by Cai-Nicolas Ziegler, Sean M. McNee, Joseph A. Konstan, Georg Lausen; describes the process of subsetting (condensation steps) the dataset as follows (p5): 

> Hence, we discarded all books missing taxonomic descriptions, along with all ratings referring to them. Next, we also removed book titles with fewer than 20 overall mentions. Only community members with at least five ratings each were kept. 

Investigate the significance of these parameters for the dataset as a whole. Additionally, decide whether to include implicit ratings (where Book-Rating equals 0) in your final dataset. Consider the potential consequences of this choice. Would you opt to exclude them prior to assessing other parameters, or would it be more appropriate to exclude them later?

Although the publication outlines the expected dimensions of the resulting dataset, it's acceptable if your findings vary at this stage.

In [12]:
subset_ratings = ratings
print(subset_ratings.shape)

subset_ratings = subset_ratings[subset_ratings["Book-Rating"] != 0]
print(subset_ratings.shape)

book_mentions = subset_ratings['ISBN'].value_counts() >= 20
subset_ratings = subset_ratings[subset_ratings['ISBN'].isin(book_mentions[book_mentions].index)]
print(subset_ratings.shape)

user_ratings = subset_ratings["User-ID"].value_counts() >= 5
subset_ratings = subset_ratings[subset_ratings['User-ID'].isin(user_ratings[user_ratings].index)]
print(subset_ratings.shape)

(1149779, 3)
(433670, 3)
(97403, 3)
(56851, 3)


### 4. Extra step
Examine the `BX-Books.csv` file specifically for the book Robots and _Empire by Isaac Asimov_. Identify any issues you come across. Would you address these issues?

Given that this could pose a problem for our dataset, consider how you would resolve it. You may need to revisit step 2 if you decide to undertake this additional step.

In [13]:
book_robots_emp = books[books["Book-Title"] == "Robots and Empire"]
print(book_robots_emp.shape)
# There are 3 publications of the book Robots and Empire by Isaac Asimov

book_robots_emp

(3, 8)


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
19463,0586062009,Robots and Empire,Isaac Asimov,1986,HarperCollins,http://images.amazon.com/images/P/0586062009.0...,http://images.amazon.com/images/P/0586062009.0...,http://images.amazon.com/images/P/0586062009.0...
83090,0345328949,Robots and Empire,Isaac Asimov,1991,Del Rey Books,http://images.amazon.com/images/P/0345328949.0...,http://images.amazon.com/images/P/0345328949.0...,http://images.amazon.com/images/P/0345328949.0...
136152,0385190921,Robots and Empire,Isaac Asimov,1985,Smithmark Pub,http://images.amazon.com/images/P/0385190921.0...,http://images.amazon.com/images/P/0385190921.0...,http://images.amazon.com/images/P/0385190921.0...


### 5. Save the new dataset(s)
Save the dataset(s) in distinct named CSV-files for later usage. Move the file(s) to the data directory.


In [14]:
subset_ratings.to_csv('data/subset_ratings.csv', sep=';')